In [0]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

In [0]:
if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')

if not os.path.exists('./vae_img_pred'):
    os.mkdir('./vae_img_pred')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 101
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [3]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
if torch.cuda.is_available():
    model.cuda()

reconstruction_function = nn.MSELoss(size_average=False)


def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [4]:
class VAEPredicter(nn.Module):
    def __init__(self):
        super(VAEPredicter, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc11 = nn.Linear(400, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc11(h1))
        return self.fc21(h2), self.fc22(h2)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)


    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return z


predModel = VAEPredicter()
if torch.cuda.is_available():
    model.cuda()

predict_function = nn.MSELoss(size_average=False)


def pred_loss_function(pred_z, target_z):#, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = predict_function(pred_z, target_z)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    #KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    #KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE


optimizer = optim.Adam(model.parameters(), lr=1e-3)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [5]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.item() / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))
model.train(mode=False)
torch.save(model.state_dict(), './vae.pth')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 0 [0/60000 (0%)]	Loss: 185.769028
Train Epoch: 0 [12800/60000 (21%)]	Loss: 51.356354
Train Epoch: 0 [25600/60000 (43%)]	Loss: 42.840378
Train Epoch: 0 [38400/60000 (64%)]	Loss: 42.023727
Train Epoch: 0 [51200/60000 (85%)]	Loss: 38.964962
====> Epoch: 0 Average loss: 45.6278
Train Epoch: 1 [0/60000 (0%)]	Loss: 36.925941
Train Epoch: 1 [12800/60000 (21%)]	Loss: 37.306335
Train Epoch: 1 [25600/60000 (43%)]	Loss: 34.015518
Train Epoch: 1 [38400/60000 (64%)]	Loss: 33.826256
Train Epoch: 1 [51200/60000 (85%)]	Loss: 33.360420
====> Epoch: 1 Average loss: 35.0348
Train Epoch: 2 [0/60000 (0%)]	Loss: 35.772167
Train Epoch: 2 [12800/60000 (21%)]	Loss: 33.084648
Train Epoch: 2 [25600/60000 (43%)]	Loss: 33.256634
Train Epoch: 2 [38400/60000 (64%)]	Loss: 33.135281
Train Epoch: 2 [51200/60000 (85%)]	Loss: 33.024170
====> Epoch: 2 Average loss: 33.1167
Train Epoch: 3 [0/60000 (0%)]	Loss: 33.344433
Train Epoch: 3 [12800/60000 (21%)]	Loss: 33.814194
Train Epoch: 3 [25600/60000 (43%)]	Loss: 

In [6]:
for epoch in range(num_epochs):
    predModel.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        mu, logvar = model.encode(img)
        modelOutput = model.reparametrize(mu, logvar)
        
        predOutput = predModel(img)
        recon_batch = model.decode(predOutput)
        predLoss = pred_loss_function(predOutput, modelOutput)
        predLoss.backward()
        train_loss += predLoss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                predLoss.item() / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img_pred/image_{}.png'.format(epoch))

torch.save(predModel.state_dict(), './vaePred.pth')

Train Epoch: 0 [0/60000 (0%)]	Loss: 39.993687


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 0 [12800/60000 (21%)]	Loss: 19.636124
Train Epoch: 0 [25600/60000 (43%)]	Loss: 21.055206
Train Epoch: 0 [38400/60000 (64%)]	Loss: 20.819111
Train Epoch: 0 [51200/60000 (85%)]	Loss: 19.913528
====> Epoch: 0 Average loss: 20.7957
Train Epoch: 1 [0/60000 (0%)]	Loss: 19.209724
Train Epoch: 1 [12800/60000 (21%)]	Loss: 20.452333
Train Epoch: 1 [25600/60000 (43%)]	Loss: 20.270781
Train Epoch: 1 [38400/60000 (64%)]	Loss: 20.551083
Train Epoch: 1 [51200/60000 (85%)]	Loss: 19.661083
====> Epoch: 1 Average loss: 20.0668
Train Epoch: 2 [0/60000 (0%)]	Loss: 19.698915
Train Epoch: 2 [12800/60000 (21%)]	Loss: 19.667227
Train Epoch: 2 [25600/60000 (43%)]	Loss: 20.088314
Train Epoch: 2 [38400/60000 (64%)]	Loss: 20.461121
Train Epoch: 2 [51200/60000 (85%)]	Loss: 20.232166
====> Epoch: 2 Average loss: 20.0199
Train Epoch: 3 [0/60000 (0%)]	Loss: 20.044392
Train Epoch: 3 [12800/60000 (21%)]	Loss: 20.265114
Train Epoch: 3 [25600/60000 (43%)]	Loss: 19.650852
Train Epoch: 3 [38400/60000 (64%)]	Lo

In [10]:
for batch_idx, data in enumerate(dataloader):
    if batch_idx == 0:
      img, _ = data
      img = img.view(img.size(0), -1)
      img = Variable(img)

      mu, logvar = model.encode(img)
      modelOutput = model.reparametrize(mu, logvar)
      
      predOutput = predModel(img)
      print(torch.max(-1 * predOutput))

tensor(3.8205, grad_fn=<MaxBackward1>)


KeyboardInterrupt: ignored